In [12]:
import pandas as pd

MAX_SPEED_METERS_PER_SECOND = 1.77

# Pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

In [13]:
df = pd.read_csv("../logs/voltage_velocity.csv")

print("Original dataframe:")
df


Original dataframe:


,time,left_voltage,right_voltage,left_speed,right_speed
0,13.64900,0.000000,0.000000,0.000000,0.000000
1,13.66900,0.000000,0.000000,0.000000,0.000000
2,13.69000,0.000000,0.000000,0.000000,0.000000
3,13.71000,0.000000,0.000000,0.000000,0.000000
4,13.73000,0.000000,0.000000,0.000000,0.000000
5,13.75000,0.000000,0.000000,0.000000,0.000000
6,13.77000,0.000000,0.000000,0.000000,0.000000
7,13.79000,0.000000,0.000000,0.000000,0.000000
8,13.81000,0.000000,0.000000,0.000000,0.000000
9,13.83000,0.000000,0.000000,0.000000,0.000000


In [ ]:
df_downsampled = df.iloc[::4].reset_index(drop=True)

In [6]:
left_df = df[["time", "left_voltage", "left_speed"]].copy()
right_df = df[["time", "right_voltage", "right_speed"]].copy()


In [7]:
left_df["left_speed"] *= (10 / MAX_SPEED_METERS_PER_SECOND)
right_df["right_speed"] *= (10 / MAX_SPEED_METERS_PER_SECOND)


In [8]:
def compute_derivatives(data, voltage_col, speed_col):
    d = data.copy()

    d["delta_time"] = d["time"].diff()
    d["delta_speed"] = d[speed_col].diff()

    # acceleration (speed change per second)
    # d["acceleration"] = d["delta_speed"] / d["delta_time"]

    return d

left_df = compute_derivatives(left_df, "left_voltage", "left_speed")
right_df = compute_derivatives(right_df, "right_voltage", "right_speed")


In [9]:
left_df["speed_bin"] = pd.cut(left_df["left_speed"], bins=range(-10, 11, 1))


In [10]:
left_df = left_df.drop("time", axis=1)
left_df = left_df.sort_values("speed_bin")


In [11]:
grouped = left_df.groupby("speed_bin")

print("\nLeft speed bins:")
bins = [bin for bin in grouped]
bins[0]



Left speed bins:


/tmp/ipykernel_19732/969070195.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = left_df.groupby("speed_bin")


(Interval(-10, -9, closed='right'),
       left_voltage  left_speed  delta_time  delta_speed  speed_bin
 188      -1.000000   -9.640096     0.01900     0.000000  (-10, -9]
 3927     -1.000000   -9.083503     0.02100     0.000000  (-10, -9]
 3941     -1.000000   -9.106989     0.02100     0.000000  (-10, -9]
 3940     -1.000000   -9.106989     0.02200    -0.041819  (-10, -9]
 3939     -1.000000   -9.065169     0.02200     0.000000  (-10, -9]
 3717     -1.000000   -9.595322     0.01899     0.000000  (-10, -9]
 3718     -1.000000   -9.595322     0.01900     0.000000  (-10, -9]
 3926     -1.000000   -9.083503     0.02100     0.000000  (-10, -9]
 1735     -0.452632   -9.729582     0.01900     0.000000  (-10, -9]
 1733     -0.663158   -9.729582     0.01900     0.000000  (-10, -9]
 1732     -0.726316   -9.729582     0.01900    -0.792667  (-10, -9]
 3938     -1.000000   -9.065169     0.02100     0.000000  (-10, -9]
 3354     -1.000000   -9.243316     0.01900     0.000000  (-10, -9]
 3368     -1